In [1]:
# Importing necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the datasets
train_path = 'data/loan_train.csv'
test_path = 'data/loan_test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Displaying the first few rows of each dataset to understand their structure
train_data_head = train_data.head()
test_data_head = test_data.head()

train_data_info = train_data.info()
test_data_info = test_data.info()

(train_data_head, test_data_head)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Co

(    Loan_ID Gender Married Dependents     Education Self_Employed  \
 0  LP001002   Male      No          0      Graduate            No   
 1  LP001003   Male     Yes          1      Graduate            No   
 2  LP001005   Male     Yes          0      Graduate           Yes   
 3  LP001006   Male     Yes          0  Not Graduate            No   
 4  LP001008   Male      No          0      Graduate            No   
 
    ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
 0             5849                0.0         NaN             360.0   
 1             4583             1508.0       128.0             360.0   
 2             3000                0.0        66.0             360.0   
 3             2583             2358.0       120.0             360.0   
 4             6000                0.0       141.0             360.0   
 
    Credit_History Property_Area Loan_Status  
 0             1.0         Urban           Y  
 1             1.0         Rural           N  
 2 

In [2]:
# Checking for missing values in train and test datasets
train_missing = train_data.isnull().sum()
test_missing = test_data.isnull().sum()

# Displaying missing values for each dataset
train_missing, test_missing


(Loan_ID               0
 Gender               13
 Married               3
 Dependents           15
 Education             0
 Self_Employed        32
 ApplicantIncome       0
 CoapplicantIncome     0
 LoanAmount           22
 Loan_Amount_Term     14
 Credit_History       50
 Property_Area         0
 Loan_Status           0
 dtype: int64,
 Loan_ID               0
 Gender               11
 Married               0
 Dependents           10
 Education             0
 Self_Employed        23
 ApplicantIncome       0
 CoapplicantIncome     0
 LoanAmount            5
 Loan_Amount_Term      6
 Credit_History       29
 Property_Area         0
 dtype: int64)

In [3]:
# Fill categorical features with the mode
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed']:
    train_data[col].fillna(train_data[col].mode()[0], inplace=True)
    test_data[col].fillna(test_data[col].mode()[0], inplace=True)

# Fill LoanAmount with median
train_data['LoanAmount'].fillna(train_data['LoanAmount'].median(), inplace=True)
test_data['LoanAmount'].fillna(test_data['LoanAmount'].median(), inplace=True)

# Fill Loan_Amount_Term with mode
train_data['Loan_Amount_Term'].fillna(train_data['Loan_Amount_Term'].mode()[0], inplace=True)
test_data['Loan_Amount_Term'].fillna(test_data['Loan_Amount_Term'].mode()[0], inplace=True)

# Fill Credit_History with mode
train_data['Credit_History'].fillna(train_data['Credit_History'].mode()[0], inplace=True)
test_data['Credit_History'].fillna(test_data['Credit_History'].mode()[0], inplace=True)

# Confirming that missing values have been handled
train_missing_after = train_data.isnull().sum()
test_missing_after = test_data.isnull().sum()

train_missing_after, test_missing_after


(Loan_ID              0
 Gender               0
 Married              0
 Dependents           0
 Education            0
 Self_Employed        0
 ApplicantIncome      0
 CoapplicantIncome    0
 LoanAmount           0
 Loan_Amount_Term     0
 Credit_History       0
 Property_Area        0
 Loan_Status          0
 dtype: int64,
 Loan_ID              0
 Gender               0
 Married              0
 Dependents           0
 Education            0
 Self_Employed        0
 ApplicantIncome      0
 CoapplicantIncome    0
 LoanAmount           0
 Loan_Amount_Term     0
 Credit_History       0
 Property_Area        0
 dtype: int64)

In [4]:
# Encoding categorical variables
label_encoder = LabelEncoder()

# Encode binary categorical variables directly
train_data['Gender'] = label_encoder.fit_transform(train_data['Gender'])
test_data['Gender'] = label_encoder.transform(test_data['Gender'])

train_data['Married'] = label_encoder.fit_transform(train_data['Married'])
test_data['Married'] = label_encoder.transform(test_data['Married'])

train_data['Education'] = label_encoder.fit_transform(train_data['Education'])
test_data['Education'] = label_encoder.transform(test_data['Education'])

train_data['Self_Employed'] = label_encoder.fit_transform(train_data['Self_Employed'])
test_data['Self_Employed'] = label_encoder.transform(test_data['Self_Employed'])

train_data['Property_Area'] = label_encoder.fit_transform(train_data['Property_Area'])
test_data['Property_Area'] = label_encoder.transform(test_data['Property_Area'])

# Encoding target variable in training data
train_data['Loan_Status'] = label_encoder.fit_transform(train_data['Loan_Status'])

# Convert "Dependents" to numerical (handling "3+" as 3)
train_data['Dependents'].replace('3+', 3, inplace=True)
test_data['Dependents'].replace('3+', 3, inplace=True)
train_data['Dependents'] = train_data['Dependents'].astype(int)
test_data['Dependents'] = test_data['Dependents'].astype(int)

# Scaling numerical features
scaler = StandardScaler()
numerical_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']

train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])
test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])

# Displaying first few rows of the processed training data to confirm changes
train_data.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,0.072991,-0.554487,-0.211241,360.0,1.0,2,1
1,LP001003,1,1,1,0,0,-0.134412,-0.038732,-0.211241,360.0,1.0,0,0
2,LP001005,1,1,0,0,1,-0.393747,-0.554487,-0.948996,360.0,1.0,2,1
3,LP001006,1,1,0,1,0,-0.462062,0.251980,-0.306435,360.0,1.0,2,1
4,LP001008,1,0,0,0,0,0.097728,-0.554487,-0.056551,360.0,1.0,2,1


In [5]:
# Splitting data into features (X) and target (y)
X = train_data.drop(columns=['Loan_ID', 'Loan_Status'])
y = train_data['Loan_Status']

# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Defining models to test
models = {
    'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced', max_iter=500),
    'Decision Tree': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced')
}

# Training and evaluating each model
results = []
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = model.predict(X_val)

    # Evaluate the model
    accuracy = accuracy_score(y_val, y_pred)
    report = classification_report(y_val, y_pred, output_dict=True)
    matrix = confusion_matrix(y_val, y_pred)

    # Store results
    results.append({
        'model': model_name,
        'accuracy': accuracy,
        'report': report,
        'confusion_matrix': matrix
    })

# Display the results
for result in results:
    print(f"Model: {result['model']}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print("Classification Report:\n", pd.DataFrame(result['report']).transpose())
    print("Confusion Matrix:\n", result['confusion_matrix'])
    print("-" * 40)


Model: Logistic Regression
Accuracy: 0.7724
Classification Report:
               precision    recall  f1-score     support
0              0.800000  0.465116  0.588235   43.000000
1              0.765306  0.937500  0.842697   80.000000
accuracy       0.772358  0.772358  0.772358    0.772358
macro avg      0.782653  0.701308  0.715466  123.000000
weighted avg   0.777435  0.772358  0.753739  123.000000
Confusion Matrix:
 [[20 23]
 [ 5 75]]
----------------------------------------
Model: Decision Tree
Accuracy: 0.7398
Classification Report:
               precision    recall  f1-score     support
0              0.648649  0.558140  0.600000   43.000000
1              0.779070  0.837500  0.807229   80.000000
accuracy       0.739837  0.739837  0.739837    0.739837
macro avg      0.713859  0.697820  0.703614  123.000000
weighted avg   0.733475  0.739837  0.734783  123.000000
Confusion Matrix:
 [[24 19]
 [13 67]]
----------------------------------------
Model: Random Forest
Accuracy: 0.7724
Cl

In [6]:
# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Initialize Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit Grid Search on the training data
grid_search.fit(X_train, y_train)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
Best Cross-Validation Score: 0.810513296227582


In [7]:
# Train Random Forest with the best parameters on the entire training data
final_rf = RandomForestClassifier(**grid_search.best_params_, random_state=42)
final_rf.fit(X, y)

# Preprocess test data and make predictions
X_test = test_data.drop(columns=['Loan_ID'])
y_test_pred = final_rf.predict(X_test)

# Prepare the output DataFrame
output = test_data[['Loan_ID']].copy()
output['Loan_Status'] = label_encoder.inverse_transform(y_test_pred)

# Save the predictions to a CSV file
output.to_csv('loan_approval_predictions.csv', index=False)
print("Predictions saved to 'loan_approval_predictions.csv'")


Predictions saved to 'loan_approval_predictions.csv'
